## INTRODUCTION TO RNN USING TENSORFLOW

#### ARCHITECTURE

![](./images/rnn-basic-image)

#### UNROLLED RNN

![](./images/rnn-unrolled)

### STEP 1: IMPORT LIBRARY AND LOAD DATASET

In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### STEP 2: CREATE AN INTERACTIVE SESSION

In [2]:
sess = tf.InteractiveSession()

### STEP 3: DEFINE THE SHAPE OF OUR DATASET

In [3]:
x = tf.placeholder(tf.float32, shape=[None, 28, 28])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

### STEP 4: DEFINE HELPER FUNCTIONS

#### 1. FOR WEIGHTS AND BIASES

In [4]:
W = lambda shape: tf.Variable(tf.truncated_normal(shape, stddev=0.1))
b = lambda shape: tf.Variable(tf.constant(0.1, shape=shape))

### STEP 5: DEFINE MODEL

#### CREATE A RNN CELL

In [5]:
W_rnn = W(shape=[128, 10])
b_rnn = b(shape=[10])
x_rnn = tf.unstack(x, 28, 1)

lstm_cell = tf.contrib.rnn.BasicLSTMCell(128, forget_bias=1.0)

outputs, states = tf.contrib.rnn.static_rnn(lstm_cell, x_rnn, dtype=tf.float32)

y_rnn = tf.matmul(outputs[-1], W_rnn) + b_rnn

### STEP 6: DEFINE METRICS

#### 1. LOSS IN THE FORM OF CROSS ENTROPY

In [6]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_rnn))

#### 2. TRAINING USING ADAM OPTIMIZER

In [7]:
global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(
    learning_rate=0.5,
    global_step=global_step,
    decay_steps=10000,
    decay_rate=0.9
)

In [8]:
train_step = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy, global_step)

#### 3. ACCURACY OF THE MODEL

In [9]:
correct_prediction = tf.equal(tf.argmax(y_rnn, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

### STEP 7: TRAIN MODEL

In [10]:
sess.run(tf.global_variables_initializer())

In [11]:
for i in range(1000):
    batch = mnist.train.next_batch(100) 
    train_step.run(feed_dict={x: batch[0].reshape(-1, 28, 28), y_: batch[1]})
    
    ### TEST OUR MODEL INBETWEEN
    if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={ x: batch[0].reshape(-1, 28, 28), y_: batch[1] })
        print('STEP', i, ':', 'ACCURACY', train_accuracy)

STEP 0 : ACCURACY 0.25
STEP 100 : ACCURACY 0.93
STEP 200 : ACCURACY 0.91
STEP 300 : ACCURACY 0.96
STEP 400 : ACCURACY 0.96
STEP 500 : ACCURACY 0.99
STEP 600 : ACCURACY 0.98
STEP 700 : ACCURACY 0.99
STEP 800 : ACCURACY 1.0
STEP 900 : ACCURACY 0.99


### STEP 8: EVALUATE MODEL

In [12]:
final_accuracy = accuracy.eval(feed_dict={ x: mnist.test.images.reshape((-1, 28, 28)) , y_: mnist.test.labels })
print('FINAL ACCURACY:', final_accuracy)

FINAL ACCURACY: 0.9693


### STEP 9: TEST PREDICTION

In [13]:
x_actual = mnist.test.images[0:1].reshape((-1, 28, 28))
y_actual = mnist.test.labels[0:1]

print('ACTUAL VALUE:', y_actual)

ACTUAL VALUE: [[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]]


In [14]:
y_predicted = sess.run(y_rnn, feed_dict={x: x_actual})

print('PREDICTED VALUE:', y_predicted)

PREDICTED VALUE: [[-2.85356975 -0.46849513  0.41614413  0.32991791 -0.63278311 -2.46221805
  -3.66423392  8.1272316  -2.99285173  0.46855268]]


In [15]:
output_actual = sess.run(tf.argmax(y_actual, 1))
output_predicted = sess.run(tf.argmax(y_predicted, 1))

print ('EXPECTED OUTPUT:', output_actual)
print ('PREDICTED OUTPUT:', output_predicted)

EXPECTED OUTPUT: [7]
PREDICTED OUTPUT: [7]
